In [81]:
import struct
import numpy as np

In [83]:
def load_sparse_data(fname):
    with open(fname, 'rb') as fd:
        magic = b''
        for b in struct.unpack('4c', fd.read(4)):
            magic += b
        if magic != b'\x00\x00\xae\xfd':
            raise RuntimeError("Filetype does not match expected for sparse data")
        
        # load array size
        size = struct.unpack('3I', fd.read(12))
        
        # load nnz
        nnz = struct.unpack('L', fd.read(8))[0]
        
        # load indices
        index = struct.unpack('{:d}L'.format(nnz), fd.read(nnz*8))
        
        # load values
        value = struct.unpack('{:d}d'.format(nnz), fd.read(nnz*8))
    return size, index, value

def sparse2dense(size, index, value):
    """convert sparse data (COO) to dense array"""
    arr = np.zeros(np.product(size))
    for idx, val in zip(index, value):
        arr[idx] = val
    return arr.reshape(size[::-1])

In [86]:
size, index, value = load_sparse_data('dose3d.bin')
dense = sparse2dense(size, index, value)

In [ ]:
import matplotlib.pyplot as plt
for ii in range(0, dense.shape[0], 2):
    plt.figure()
    plt.imshow(dense[ii])
    plt.show()
